In [7]:
import pandas as pd
from IPython.display import display, HTML
import os

pd.set_option('expand_frame_repr', False)

display(HTML(data="""
<style>
    div#notebook-container    { width: 90%; }                                                                                        
    div#menubar-container     { width: 90%; }
</style>
"""))

In [8]:
drug_dic = {'marij': 'Marijuana',
       'cocaine': 'Cocaine',
       'crack': 'Crack',
       'heroin': 'Heroin',
       'hallucinogen': 'Hallucinogen',
       'inhalant':'Inhalant',
       'meth': 'Meth',
       'painrelieve': 'PainReliever',
       'tranq': 'Tranquilizer',
       'stimulant':'Stimulant',
       'sedative':'Sedative',
       'anydrug': 'any_drug',
        'countofdrugs':'countofdrugs'}

user_type_dic = {0: 'Non User',
                 1: 'Light User',
                 2: 'User',
                 3: 'Heavy User'}

education_dic = {1: "Less than High School",
        2: "High School Graduate",
        3: "Some College Credit",
        4: "College Graduate"}

sex_dic = {1: "Male", 2: "Female"}
    
employment_dic = {1: "Full-Time", 2: "Part-Time", 3: "Unemployed"}

file = pd.read_csv('NSDUH-Workforce-AdultsCorrect.csv', delimiter=",")
file["SelectiveLeave"] = file["SelectiveLeave"]//7
file["SkipSick"] = file["SkipSick"]//7

file.loc[file["SelectiveLeave"] == 0,"SelectiveLeave"] = 1
file.loc[file["SkipSick"] == 0,"SkipSick"] = 1
columns = file.columns


In [9]:
def get_columns(substring,columns):
    cols = []

    for column in columns:
        if substring in column:
            cols.append(column)
    return cols

In [7]:
import pandas as pd

doc = pd.DataFrame()
index = 0

for column in get_columns("user_type",columns):
    column_name = drug_dic[column.replace("_user_type","")]
    doc.insert(index,column_name,file.groupby([column]).size()/file.shape[0])
    index += 1

doc = doc.T
doc.to_csv('user_type_count.csv')

In [91]:
Xdrug_ever = file[get_columns("_ever",columns)]
doc = pd.DataFrame()
index = 0

for column in Xdrug_ever.columns:
    yes_ever = Xdrug_ever.query(column + "==1")
    doc.insert(index,column,yes_ever.sum()/yes_ever.shape[0])
    index += 1

doc.to_csv('drugs_consuming_assoc.csv')

In [20]:
demo_info = ["SelectiveLeave","PersonalIncome","Education","EmploymentStatus"]
countings = {}

for column in get_columns("_user_type",columns):
    drug_name = drug_dic[column.replace("_user_type","")]
    for demo in demo_info:
        doc = file[[column,demo]]
        doc = doc.rename(index=str,columns={column: "UserType", demo: "DemoInfo"})
        doc["Distribution"] = 1
        
        totalInquiries = len(doc)
        groupby = doc.groupby(["UserType","DemoInfo"]).sum()/totalInquiries
    
        groupby.to_csv(drug_name+demo+".csv")

In [66]:
import glob
import math

path = "."
files = glob.glob(os.path.join(path,"*.csv"))

for file in files:
    filename = os.path.splitext(os.path.basename(file))[0]
    if "DemoInfo" in filename or "count" in filename or "titanic" in filename or "Correct" in filename or "assoc" in filename:
        continue
    print(filename)

    df = pd.read_csv(file)
    cols = df.columns

    df = df.apply(lambda x: [x['UserType'],x[cols[1]],math.floor((x["Distribution"]/countings[filename][x['UserType']])*100)], axis=1)
    df.to_csv(filename + ".csv",index=False)

CocaineEducation
CocaineEmploymentStatus
CocainePersonalIncome
CocaineSelectiveLeave
CrackEducation
CrackEmploymentStatus
CrackPersonalIncome
CrackSelectiveLeave
HallucinogenEducation
HallucinogenEmploymentStatus
HallucinogenPersonalIncome
HallucinogenSelectiveLeave
HeroinEducation
HeroinEmploymentStatus
HeroinPersonalIncome
HeroinSelectiveLeave
InhalantEducation
InhalantEmploymentStatus
InhalantPersonalIncome
InhalantSelectiveLeave
MarijuanaEducation
MarijuanaEmploymentStatus
MarijuanaPersonalIncome
MarijuanaSelectiveLeave
MethEducation
MethEmploymentStatus
MethPersonalIncome
MethSelectiveLeave
PainRelieverEducation
PainRelieverEmploymentStatus
PainRelieverPersonalIncome
PainRelieverSelectiveLeave
SedativeEducation
SedativeEmploymentStatus
SedativePersonalIncome
SedativeSelectiveLeave
StimulantEducation
StimulantEmploymentStatus
StimulantPersonalIncome
StimulantSelectiveLeave
TranquilizerEducation
TranquilizerEmploymentStatus
TranquilizerPersonalIncome
TranquilizerSelectiveLeave


In [6]:
demo_info = ["Sex","Education","EmploymentStatus"]

for column in get_columns("_ever",columns):
    drug_name = drug_dic[column.replace("_ever","")]
    yes_ever = file.query(column + "==1 and Race in ([1,2,6,7])")
    doc = yes_ever[demo_info]
    doc = doc.rename(index=str, columns={"EmploymentStatus": "Employment Status"})
        
    doc["Sex"] = doc["Sex"].apply(lambda x: sex_dic[x])
    doc["Education"] = doc["Education"].apply(lambda x: education_dic[x])
    doc["Employment Status"] = doc["Employment Status"].apply(lambda x: employment_dic[x])
    doc.to_csv(drug_name+"DemoInfo.csv",index=False)